In [87]:
import pymupdf
import pymupdf4llm
import pandas as pd
import glob
import re

In [88]:
all_pdf = glob.glob(r"PDF2\*.pdf")

In [89]:
pdf_document = r"PDF2\Об инциденте с NTP-серверами _ Хабр.pdf"
doc = pymupdf.open(pdf_document)
print("Исходный документ: ", doc.name)
print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
print(doc.metadata)
text = []
for current_page in range(len(doc)):
    page = doc.load_page(current_page)
    page_text = page.get_text("text")
    print("Стр. ", current_page + 1, "\n")
    text.append(page_text)
    print(page_text)

Исходный документ:  PDF2\Об инциденте с NTP-серверами _ Хабр.pdf

Количество страниц: 5

------------------


{'format': 'PDF 1.4', 'title': 'Об инциденте с NTP-серверами / Хабр', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0', 'producer': 'Skia/PDF m136', 'creationDate': "D:20250505174507+00'00'", 'modDate': "D:20250505174507+00'00'", 'trapped': '', 'encryption': None}
Стр.  1 

680.84
Рейтинг
Яндекс
Как мы делаем Яндекс
Подписаться
Недавно на Хабре вышла статья об аномальной нагрузке на публичные NTP‑серверы
в рунете. К сожалению, к этой ситуации привела ошибка в прошивке Яндекс Станций.
Как наши Станции синхронизируют время, что спровоцировало инцидент и как мы
планируем поступить дальше — обо всём этом расскажем в статье. Хочется верить,
что описание ситуации и принятые меры помогут не только нам, но и коллегам
по индустрии избежать подобного в бу

In [90]:
def extract_markdown_text(document) -> str:
    return pymupdf4llm.to_markdown(document)
    

In [91]:
print(extract_markdown_text(pdf_document))

05.05.2025, 22:45 Об инциденте с NTP-серверами / Хабр

[КАК СТАТЬ АВТОРОМ](https://habr.com/ru/sandbox/start/)
#### **680.84**

Рейтинг
## **Яндекс**

Как мы делаем Яндекс

Подписаться

**[slavashel](https://habr.com/ru/users/slavashel/)**

27 ноя 2024 в 07:54
# Об инциденте с NTP-серверами

**5 мин** **82K**
#### Блог компании Яндекс,  Сетевые технологии*,  Высокая производительность*,  IT-инфраструктура*

Ретроспектива
#### Недавно на Хабре вышла статья об аномальной нагрузке на публичные NTP ‑ серверы в рунете. К сожалению, к этой ситуации привела ошибка в прошивке Яндекс Станций. Как наши Станции синхронизируют время, что спровоцировало инцидент и как мы планируем поступить дальше — обо всём этом расскажем в статье. Хочется верить, что описание ситуации и принятые меры помогут не только нам, но и коллегам по индустрии избежать подобного в будущем.

Динамика числа отправляемых колонками NTP-запросов
## Как Станции обновляют время

https://habr.com/ru/companies/yandex/articles/861538

In [92]:
def parse_markdown(document: str):
    print(document)

In [93]:
def parse_article(text):
    lines = text.splitlines()
    result = {}
    for line in lines:
        m = re.match(r"^#### \*\*(?P<rating>[\d.]+)\*\*$", line)
        if m:
            result['rating'] = m.group('rating')
            break
    for i, line in enumerate(lines):
        m = re.match(r"^## \*\*(?P<company>.+)\*\*$", line)
        if m:
            result['company_name'] = m.group('company').strip()
            for desc in lines[i+1:]:
                if desc.strip():
                    result['company_description'] = desc.strip()
                    break
            break
    m = re.search(r"\*\*\[(?P<author>[^\]]+)\]\([^)]+\)\*\*", text)
    if m:
        result['author'] = m.group('author')
    m = re.search(r"\b(\d{1,2}\s+\w+\s+\d{4}\s+в\s+\d{2}:\d{2})\b", text)
    if m:
        result['publish_date'] = m.group(1)
    m = re.search(r"^# (?P<title>.+)$", text, flags=re.MULTILINE)
    if m:
        result['title'] = m.group('title').strip()
        result['article_text'] = text[m.end():].strip()
    return result

In [94]:
Brak = []
Author = []
PublishDate = []
Rating = []
ArticleText = []
CompanyName = []
CompanyDescription = []
for pdf in all_pdf:
    try:  
        text = extract_markdown_text(pdf)
        result = parse_article(text)

        print(result)

        author = result["author"]
        publish_date = result["publish_date"]
        rating = result["rating"]
        article_text = result["article_text"]
        company_name = result["company_name"]
        # company_description = result["description"]

        Author.append(author)
        PublishDate.append(publish_date)
        Rating.append(rating)
        ArticleText.append(article_text)
        CompanyName.append(company_name)
        # CompanyDescription.append(company_description)
    except:
        text = extract_markdown_text(pdf)
        print("Ошибка в файле: ", pdf)
        print(text)
        # Brak.append(pdf)

{'rating': '253', 'company_name': 'InlyIT', 'company_description': 'Для старательного нет ничего невозможного', 'author': 'InlyIT', 'publish_date': '31 мая 2024 в 07:23', 'title': 'Cloudflare положил наш сайт после того, как мы отказались выплатить 120 000 $ в течение 24 часов', 'article_text': '**9 мин** **131K**\n##### Блог компании InlyIT,  Администрирование доменных имен*,  Облачные сервисы*\n\nПеревод\n\n[Автор оригинала: Robin Dev](https://robindev.substack.com/p/cloudflare-took-down-our-website)\n##### Я работаю системным оператором в довольно крупном онлайн-казино (мне представляется, что материал статьи может быть полезен читателям вне зависимости от их позиции касательно этичности казино в целом – упоминаю об этом просто для контекста). У нас около четырех миллионов активных пользователей в месяц. С Cloudflare мы охотно сотрудничаем с 2018 года по тарифу «Бизнес», который стоит 250 $ в месяц и предоставляет неплохие возможности, включая безлимитный трафик. Нужно признать, 250

In [ ]:
Brak

[]

In [ ]:
from numpy import float32


# df = pd.DataFrame({
# 'Author': pd.Series(dtype=str),
# 'PublishDate': pd.Series(dtype=float32),
# 'DataPublish': pd.Series(dtype=str),
# 'Activity': pd.Series(dtype=str),
# 'TextArticle': pd.Series(dtype=str)
# })

df = pd.DataFrame({
    'Author': pd.Series(dtype=str),
    'PublishDate': pd.Series(dtype=str),
    'Rating': pd.Series(dtype=float32),
    'ArticleText': pd.Series(dtype=str),
    'CompanyName': pd.Series(dtype=str),
    'CompanyDescription': pd.Series(dtype=str)
})

In [ ]:
df['Author'] = Author
df['PublishDate'] = PublishDate
df['Rating'] = Rating
df['ArticleText'] = ArticleText
df['CompanyName'] = CompanyName
df['CompanyDescription'] = CompanyDescription

['InlyIT', 'ru_vds', 'km1337', 'stalkermustang', 'ChrisVechkanova', 'ru_vds', '3_15_10_20', 'ntsaplin', 'shiru8bit', 'mmamayka', 'OldFashionedEngineer', 'ru_vds', 'ntsaplin', 'evgen_ru', 'Asxzay', 'shvedov_grangroup', 'shiru8bit', 'IlyaDziub', 'stalkermustang', 'slavashel', 'glaschenko', 'MaFrance351', 'Alkash-kolyadun', '15432', 'kesn', 'apoltavcev', 'DandyDan', 'artyomsoft', 'AzazKamaz', 'vetas77', 'ptsecurity', 'Basis_Habr', 'GreyTomcat', 'engine9', 'Antxak']


In [ ]:
df

,Author,PublishDate,Rating,ArticleText,CompanyName,CompanyDescription
0,InlyIT,"05.05.2025, 23:11",400.0,Перевод\n\n[Автор оригинала: Robin Dev](https:...,Хабр,+249
1,ru_vds,"06.05.2025, 08:11",31000.0,Обзор Перевод\n\n[Автор оригинала: Avinash Saj...,Хабр,Средний
2,km1337,"05.05.2025, 22:56",124.0,,Хабр,+262
3,stalkermustang,"06.05.2025, 08:14",67000.0, \n\nОбзор\n\n##### ChatGPT вышел уже почти ...,Хабр,Средний
4,ChrisVechkanova,"05.05.2025, 22:48",327.0,https://habr.com/ru/companies/greenatom/articl...,Хабр,+283
5,ru_vds,"05.05.2025, 23:21",60000.0, \n\nОбзор Перевод\n\n[Автор оригинала: Tim]...,Хабр,Простой
6,3_15_10_20,"05.05.2025, 23:28",307.0, \n\n[✏✏️ Технотекст 7](https://habr.com/ru/...,Хабр,+200
7,ntsaplin,"05.05.2025, 22:47",249.0,[✏✏️ Технотекст 7](https://habr.com/ru/technot...,Хабр,+322
8,shiru8bit,"05.05.2025, 23:31",27000.0," \n\nОбзор\n\n##### Как говорится, вместо ты...",Хабр,Простой
9,mmamayka,"05.05.2025, 23:17",47000.0, \n\nКейс [✏✏️ Технотекст 7](https://habr.co...,Хабр,Сложный


In [ ]:
# pdf = extract_text_from_pdf(r"PDF2\ASML_ Создание самых сложных машин на Земле _ Хабр.pdf")

In [ ]:
# pars_pdf(pdf[0])